<a href="https://colab.research.google.com/github/yusam/mead2/blob/main/autos_de_infracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carregando bibliotecas

In [224]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
from google.colab import drive

### Montando drive do Google Drive

In [ ]:
drive.mount('/content/drive')

### Carregando dataset


In [ ]:
infracao_aux_2018 = []
infracao_aux_2019 = []
infracao_aux_2020 = []


#le todos os arquivos de 2020
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2020/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    infracao_aux.columns=["tipo_infracao",
                          "descricao",
                          "tipo_infrator",
                          "tipo_veiculo",
                          "data_cometimento",
                          "hora_cometimento",
                          "local_rodovia",
                          "local_km",
                          "local_referencia",
                          "local_complemento",
                          "latitude",
                          "longitude",
                          "gravidade"]
    infracao_aux_2020.append(infracao_aux)

    
   
#le todos os arquivos de 2019
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2019/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    infracao_aux.columns=["tipo_infracao",
                          "descricao",
                          "tipo_infrator",
                          "tipo_veiculo",
                          "data_cometimento",
                          "hora_cometimento",
                          "local_rodovia",
                          "local_km",
                          "local_referencia",
                          "local_complemento",
                          "latitude",
                          "longitude",
                          "gravidade"]
    infracao_aux_2019.append(infracao_aux)


#le todos os arquivos de 2018
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2018/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    if infracao_aux.shape[1] == 6:
        infracao_aux.columns=["tipo_infracao",
                              "descricao",
                              "tipo_infrator",
                              "tipo_veiculo",
                              "data_cometimento",
                              "hora_cometimento",]
    else:
        infracao_aux.columns=["tipo_infracao",
                              "descricao",
                              "tipo_infrator",
                              "tipo_veiculo",
                              "data_cometimento",
                              "hora_cometimento",
                              "local_rodovia",
                              "local_km",
                              "local_referencia",
                              "local_complemento",
                              "latitude",
                              "longitude",
                              "gravidade"]
    infracao_aux_2018.append(infracao_aux)    

In [ ]:
## juntando dataframes de anos difrentes
infracao1 = pd.concat(infracao_aux_2018,ignore_index = True)
infracao2 = pd.concat(infracao_aux_2019,ignore_index = True)
infracao3 = pd.concat(infracao_aux_2020,ignore_index = True)

infracao = infracao1
infracao = infracao.append(infracao2)
infracao = infracao.append(infracao3)

del infracao1, infracao2, infracao3

infracao.head().T


In [ ]:
## verificando tipo de variaveis
infracao.info()

In [ ]:
## convertendo para data
infracao['data_cometimento'] = pd.to_datetime(infracao['data_cometimento'])
infracao.info()

In [ ]:
## verificando qtd de linhas e colunas
infracao.shape

### Limpeza dos dados

#### Tratando valores duplicados

In [ ]:
## verificando valores duplicados
infracao.duplicated().value_counts()

In [ ]:
## dropando valores duplicados
infracao.drop_duplicates(inplace=True)
infracao.shape

#### Tratando valores nulos

In [ ]:
## verificando a quantidade de nulos
infracao.isna().sum().sort_values()

In [ ]:
## Convertendo campos para nulos - tipo de veiculo
infracao.loc[:,"tipo_veiculo"] = (infracao.loc[:,"tipo_veiculo"]                              
                                  .str.strip() .replace("-",np.nan)
                                  .replace("null",np.nan)
                                  .replace("INEXISTENTE",np.nan)                                                        
                                  .str.upper()
                                  )

In [ ]:
## verificando a quantidade de nulos
infracao.isna().sum().sort_values()

In [ ]:
## dropando colunas nulas não utilizadas
infracao = infracao.drop(columns=['longitude','latitude','local_complemento','local_referencia','local_km','local_rodovia'])
infracao.isnull().sum().sort_values()

In [ ]:
## tratando valores nulos
## utilizando a moda para os tipos de veiculos 
infracao['tipo_veiculo'].fillna(value = infracao['tipo_veiculo'].mode().to_string(), inplace=True)
infracao.isnull().sum().sort_values()

#### Normalizando dados

In [ ]:
## padronizando tipo infracao
infracao.loc[:,"tipo_infracao"] = infracao.loc[:,"tipo_infracao"].str.strip()
infracao.tipo_infracao.value_counts()

In [ ]:
## padronizando tipo infrator
infracao.loc[:,"tipo_infrator"] = (infracao.loc[:,"tipo_infrator"]
                                   .str.strip()    
                                   .str.replace("(Proprietário|ProprietÃ¡rio)","PROPRIETARIO")
                                   .str.replace("(Pessoa Jurídica|Pessoa JurÃ\xaddica)","PESSOA_JURIDICA")
                                   .str.replace("(Pessoa Física|Pessoa FÃ\xadsica)","PESSOA_FISICA")
                                   .str.replace("Servidor Público","SERVIDOR_PUBLICO")                               
                                   .str.upper()
                                   )
infracao.tipo_infrator.value_counts()

In [ ]:
## padronizando gravidade da infracao
infracao.loc[:,"gravidade"] = (infracao.loc[:,"gravidade"]
                               .str.strip()
                               .str.replace("(MÃ©dia)","MEDIA")
                               .str.replace("GravÃ\xadssima","GRAVISSIMA")                           
                               .str.upper()
                               )
infracao.gravidade.value_counts()

In [ ]:
## padronizando nome tipo de veiculo
infracao.loc[:,"tipo_veiculo"] = (infracao.loc[:,"tipo_veiculo"]    
                                  .str.replace("(CARRO-DE).*","CARRO_DE_MAO") 
                                  .str.replace("(AUTOM|0    AUTO).*","AUTOMOVEL")   
                                  .str.replace("(MOTOCICLET).*","MOTOCICLETA")
                                  .str.replace("(CAMINHONET).*","CAMINHONETE")
                                  .str.replace("(CAMINH)[^o|^O].[^\s]*|C[\.]","CAMINHAO")
                                  .str.replace("(\sTRATOR|\sTrator)","_TRATOR")
                                  .str.replace("(UTILI).*","UTILITARIO")
                                  .str.replace("(TRATOR).*","TRATOR_DE_RODAS")
                                  .str.replace("(.NIBU|Ã\x94).*","ONIBUS")
                                  .str.replace("(MICR|Ã\x94).*","MICROONIBUS")
                                  .str.replace("[TRIC](ICLOM).*","CICLOMOTOR")
                                  .str.replace("(SEMI|S\.)(-|)","SEMI")
                                  .str.replace("(MOTOR CASA|MOTOR-CASA)","TRAILER")
                                  .str.replace("CHASSI PLATAFORMA","CHASSI_PLATAFORMA")
                                  .str.replace("TR RODAS","TRANSP_RODAS")
                                  .str.replace("TR MISTO","TRANSP_MISTO")
                                  .str.replace("TR ESTEIRAS","TRANSP_ESTEIRAS")  
                                  )
infracao.tipo_veiculo.value_counts()

#### tratando campo gravidade

In [ ]:
infracao.gravidade.value_counts(normalize=True)

In [ ]:
infracao.isnull().sum().sort_values()

In [ ]:
## tratando valores nulos
## utilizando a moda para os tipos de veiculos 
infracao['gravidade'].fillna(value = infracao['gravidade'].mode().to_string(), inplace=True)
infracao.isnull().sum().sort_values()

In [ ]:
## padronizando gravidade da infracao
infracao.loc[:,"gravidade"] = (infracao.loc[:,"gravidade"]
                               .str.strip()
                               .str.replace("(MÃ©dia|0    MEDIA)","MEDIA")
                               .str.replace("GravÃ\xadssima","GRAVISSIMA")                           
                               .str.upper()
                               )
infracao.gravidade.value_counts(normalize=True)

### Gerando arquivo tratado

In [225]:
## exportando resultado
infracao.to_csv('/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/infracoes_2018_2020_tratado.csv', sep=';',encoding="latin-1")